In [ ]:
using InteractiveUtils; versioninfo()
const RESEARCH_DIR = dirname(@__DIR__)
using Pkg; Pkg.activate(RESEARCH_DIR); Pkg.resolve(); Pkg.instantiate(); Pkg.precompile(); Pkg.status()

In [2]:
# using Revise

maybeincludet(args...; kwargs...) = 
    (isdefined(Main, :Revise) ? includet(args...; kwargs...) : include(args...; kwargs...))

using PyCall, DrWatson
maybeincludet(joinpath(RESEARCH_DIR, "src", "experiment_utility.jl"))

using LinearAlgebra, Statistics, AdvancedHMC
using AdvancedHMC: AbstractKinetic
using AdvancedHMC.Experimental
using AdvancedHMC.Experimental: AbstractRelativisticKinetic, relativistic_energy, relativistic_mass, relativistic_energy

maybeincludet(joinpath(RESEARCH_DIR, "src", "sampler_utility.jl"))
maybeincludet(joinpath(RESEARCH_DIR, "src", "geweke_utility.jl"))

using Logging: NullLogger, with_logger

In [ ]:
function plot_with_initial!(ax, samples_x, samples_y)
    num_samples = length(samples_x)
    ax.scatter(samples_x,    samples_y,    s=8.0,      c=(collect(1:num_samples) / num_samples))
    ax.scatter(samples_x[1], samples_y[1], marker="x", c="red", label="\$q_0\$")
end


hps = (; target=:gaussian, n_samples=2_000, metric=:dense_euclidean, λ=1e-2, α=20.0, m=0.2, c=20.0, integrator=:lf, ϵ=0.05, n=6, L=8)

@time retval = with_logger(NullLogger()) do 
    sample_target(
        # (; hps..., target=:funnel, kinetic=:relativistic, integrator=:lf, ϵ=0.2)
        # (; hps..., target=:funnel, kinetic=:relativistic, integrator=:glf, ϵ=0.2)
        # (; hps..., target=:funnel, metric=:dense_riemannian_softabs, kinetic=:gaussian, integrator=:glf)
        (; hps..., target=:funnel, metric=:dense_riemannian_softabs, kinetic=:gaussian, integrator=:glf, ϵ=0.25, L=32, n=12)
    )
end

@info "Average acceptance ratio" mean(map(s -> s.is_accept, retval.stats))

let (fig, ax) = subplots()
    
    samples_x = map(s -> s[1], retval.samples)
    samples_y = map(s -> s[2], retval.samples)

    plot_with_initial!(ax, samples_x, samples_y)

    ax.legend()
    
    fig
end

AD check for `∂H∂θ` and `∂H∂r`

In [ ]:
using FiniteDiff: finite_difference_gradient, finite_difference_hessian, finite_difference_jacobian
using AdvancedHMC: neg_energy, energy, ∂H∂θ, ∂H∂r

function check(a, b)
    @show a
    @show b
    @show maximum(abs.(a - b))
    @show norm(a - b, 2)
end

let hps = (; λ=1e-2, α=20.0, ϵ=0.1, n=6, L=8)
    
    target = Funnel()

    rng = MersenneTwister(1110)

    θ₀ = randn(rng, dim(target))

    ℓπ = VecTargets.gen_logpdf(target)
    ∂ℓπ∂θ = VecTargets.gen_logpdf_grad(target, θ₀)

    Vfunc, Hfunc, Gfunc, ∂G∂θfunc = prepare_sample_target(hps, θ₀, ℓπ)

    D = dim(target) # ==2 for this test
    x_zero = zeros(D)
    x_rand = randn(rng, D)
    x = x_rand
    r = randn(rng, D)
    
    # metric = UnitEuclideanMetric(D)
    # metric = DiagEuclideanMetric(D)
    # metric = DiagEuclideanMetric([0.5, 4.0])
    # metric = DenseEuclideanMetric((D,))
    # metric = DenseEuclideanMetric([1.32 0.79; 0.79 0.48])
    metric = DenseRiemannianMetric((D,), Gfunc, ∂G∂θfunc, SoftAbsMap(hps.α))
    
    # kinetic = GaussianKinetic()
    kinetic = RelativisticKinetic(0.1, 10.0)
    
    hamiltonian = Hamiltonian(metric, kinetic, ℓπ, ∂ℓπ∂θ)
    
    Hamifunc = (x, r) -> energy(hamiltonian, r, x) + energy(hamiltonian, x)
    Hamifuncx = x -> Hamifunc(x, r)
    Hamifuncr = r -> Hamifunc(x, r)
    
    check(finite_difference_gradient(Hamifuncx, x), ∂H∂θ(hamiltonian, x, r).gradient)
    
    check(finite_difference_gradient(Hamifuncr, r), ∂H∂r(hamiltonian, x, r))
    
    # dimensionwise_kinetic = DimensionwiseRelativisticKinetic(0.1, 10.0)
    # dimensionwise_hamiltonian = Hamiltonian(metric, dimensionwise_kinetic, ℓπ, ∂ℓπ∂θ)
    
    # Hamifunc = (x, r) -> energy(dimensionwise_hamiltonian, r, x) + energy(dimensionwise_hamiltonian, x)
    # Hamifuncx = x -> Hamifunc(x, r)
    # Hamifuncr = r -> Hamifunc(x, r)
    
    # check(finite_difference_gradient(Hamifuncx, x), ∂H∂θ(dimensionwise_hamiltonian, x, r).gradient)
    
    # check(finite_difference_gradient(Hamifuncr, r), ∂H∂r(dimensionwise_hamiltonian, x, r))
end

;

Check energy preservation in numerical integration

In [ ]:
@time retval = with_logger(NullLogger()) do 
    sample_target(
        (; hps..., target=:funnel, metric=:dense_riemannian_softabs, kinetic=:relativistic, integrator=:glf); 
        # (; hps..., target=:funnel, metric=:dense_riemannian_softabs, kinetic=:dimwise_relativistic, integrator=:glf);
        rng=MersenneTwister(1110),
    )
end

@info "Average acceptance ratio" mean(map(s -> s.is_accept, retval.stats))

let (fig, ax) = subplots()
    
    let num_samples = length(retval.samples), 
        samples_x = map(s -> s[1], retval.samples),
        samples_y = map(s -> s[2], retval.samples)

        plot_with_initial!(ax, samples_x, samples_y)
    end

    ax.legend()
    
    fig
end

In [ ]:
@time retval_lst = with_logger(NullLogger()) do 
    map(1:3) do seed_shift
        sample_target(
            (; hps..., target=:funnel, metric=:dense_riemannian_softabs, kinetic=:relativistic, integrator=:glf); 
            rng=MersenneTwister(1110 + seed_shift),
        )
    end
end

let (fig, ax) = subplots(figsize=(12, 4))
    
    for (seed_shift, retval) in enumerate(retval_lst)
        ax.plot(map(s -> s.hamiltonian_energy, retval.stats), label="seed=$(1110+seed_shift)")
    end

    ax.legend()
    
    fig
end

Geweke tests

In [ ]:
using MCMCDebugging
using Plots: Plots

make_kinetic(hps) = @match hps.kinetic begin
    :gaussian => GaussianKinetic()
    :relativistic => RelativisticKinetic(hps.m, hps.c)
    :dimwise_relativistic => DimensionwiseRelativisticKinetic(hps.m, hps.c)
end

g(θ, x) = cat(θ, x; dims=1)

In [ ]:
function rand_θ_given_basic(hps, target, x_data; seed=rand(UInt32))
    @info "" hps...
    
    rng = MersenneTwister(seed)
    
    D = dim(target) - 1
    θ₀ = rand(rng, D)
    
    ℓπ = x -> logpdf(target, vcat(x, x_data))
    ∂ℓπ∂θ = VecTargets.gen_grad(ℓπ, θ₀)

    # metric = UnitEuclideanMetric(D)
    # metric = DiagEuclideanMetric(D)
    # metric = DiagEuclideanMetric([0.5, 4.0])
    # metric = DenseEuclideanMetric(D)
    metric = DenseEuclideanMetric([1.32 0.79; 0.79 0.48])

    kinetic = make_kinetic(hps)
    
    hamiltonian = Hamiltonian(metric, kinetic, ℓπ, ∂ℓπ∂θ)

    TS = EndPointTS
    
    integrator = Leapfrog(hps.ϵ)
    # integrator = GeneralizedLeapfrog(hps.ϵ, hps.n)

    tc = FixedNSteps(hps.L)
    
    proposal = HMCKernel(Trajectory{TS}(integrator, tc))

    samples, stats = sample(
        rng, hamiltonian, proposal, θ₀, hps.n_samples; progress=false, verbose=true
    )
    
    return samples[end]
end

make_rand_θ_given_basic(hps, target) = x -> rand_θ_given_basic(hps, target, x)

hps_geweke = (; n_samples=200, L=16, m=0.2, c=20.0)

for (name, ϵ, kinetic) in [
    (   "HMC", 0.15, :gaussian),
    ("SR-HMC", 0.1,  :relativistic),
]
    @time res_hmc = with_logger(NullLogger()) do
        perform(GewekeTest(500), TuringFunnel, make_rand_θ_given_basic((; hps_geweke..., ϵ, kinetic), Funnel(3)); g=g)
    end

    let en = "geweke"
        p = Plots.plot(res_hmc, TuringFunnel(); size=(300, 300), title="$name sampler")
        display(p)
        ensure_path(resultsdir(en))
        Plots.savefig(p, resultsdir(en, "$name.png"))
    end
end

In [ ]:
# I still need to use the hand-written one because of AD
function rand_θ_given(hps, target, x_data; seed=rand(UInt32))
    rng = MersenneTwister(seed)
    
    D = dim(target) - 1
    θ₀ = rand(rng, D)
    
    ℓπ = x -> logpdf(target, vcat(x, x_data))
    ∂ℓπ∂θ = VecTargets.gen_grad(ℓπ, θ₀)
    
    _, _, Gfunc, ∂G∂θfunc = prepare_sample_target(hps, θ₀, ℓπ)

    metric = DenseRiemannianMetric((D,), Gfunc, ∂G∂θfunc, SoftAbsMap(hps.α))

    kinetic = make_kinetic(hps)

    hamiltonian = Hamiltonian(metric, kinetic, ℓπ, ∂ℓπ∂θ)

    TS = EndPointTS
    
    integrator = GeneralizedLeapfrog(hps.ϵ, hps.n)

    tc = FixedNSteps(hps.L)
    
    proposal = HMCKernel(Trajectory{TS}(integrator, tc))

    samples, stats = sample(
        rng, hamiltonian, proposal, θ₀, hps.n_samples; progress=false, verbose=true
    )
    
    return samples[end]
end

make_rand_θ_given(hps, target) = x -> rand_θ_given(hps, target, x)

hps_geweke = (; n_samples=200, n=6, λ=1e-2, α=20.0, m=0.5, c=4.0)

for (name, hps_extra) in [
    ("RHMC", (; ϵ=0.025, L=16, kinetic=:gaussian)), 
    ("GR-HMC", (; ϵ=0.05, L=16, kinetic=:relativistic)),
]
    @time res_rhmc = with_logger(NullLogger()) do 
        perform(GewekeTest(500), TuringFunnel, make_rand_θ_given((; hps_geweke..., hps_extra...), Funnel(3)); g=g)
    end

    let en = "geweke"
        p = Plots.plot(res_rhmc, TuringFunnel(); size=(300, 300), title="$name sampler")
        display(p)
        ensure_path(resultsdir(en))
        Plots.savefig(p, resultsdir(en, "$name.png"))
    end
end